<a href="https://colab.research.google.com/github/tractorjuice/BSHR_Loop/blob/main/demo02/Building_David_Shapiro_AI_Body_of_Knowledge_Part_1_Collect_YouTube_Videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# David Shapiro AI Body of Knowledge Using Langchain & OpenAI
## Part 1, collect the required data for processing

This example shows how to create and query an internal knowledge base using ChatGPT.

This does not require a GPU runtime.

## Set Up


###Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

KB_FOLDER = "/content/gdrive/Shareddrives/AI/DavidShapiroKB"  # Google drive folder to save the knowledgebase
YT = os.path.join(KB_FOLDER, "youtube")  # Sub-directory for YouTube FAIS datastore files
YT_DATASTORE = os.path.join(YT, "datastore")  # Sub-directory for YouTube FAIS datastore files
YT_AUDIO = os.path.join(YT, "audio")  # Sub-directory for audio files
YT_TRANSCRIPTS = os.path.join(YT_AUDIO, "transcripts")  # Sub-directory for transcripts of audio files
YT_TRANSCRIPTS_TEXT = os.path.join(YT_TRANSCRIPTS, "full_text")  # Sub-directory for text of audio files
YT_TRANSCRIPTS_WHISPER = os.path.join(YT_TRANSCRIPTS, "whisper_chunks")  # Sub-directory for Whisper chunks of audio files
YT_TRANSCRIPTS_DATASTORE = os.path.join(YT_TRANSCRIPTS, "datastore")  # Sub-directory for books FAIS datastore file
PODCAST = os.path.join(KB_FOLDER, "podcast")  # Sub-directory for YouTube FAIS datastore files
PODCAST_DATASTORE = os.path.join(PODCAST, "datastore")  # Sub-directory for YouTube FAIS datastore files
PODCAST_AUDIO = os.path.join(PODCAST, "audio")  # Sub-directory for YouTube FAIS datastore files
PODCAST_TRANSCRIPTS = os.path.join(PODCAST, "transcripts")  # Sub-directory for YouTube FAIS datastore files
PODCAST_TRANSCRIPTS_TEXT = os.path.join(PODCAST_TRANSCRIPTS, "full_text")  # Sub-directory for text of audio files
PODCAST_TRANSCRIPTS_WHISPER = os.path.join(PODCAST_TRANSCRIPTS, "whisper_chunks")  # Sub-directory for Whisper chunks of audio files

# Check if directory exists and if not, create it
if not os.path.exists(KB_FOLDER):
    os.makedirs(KB_FOLDER)

# Check if directory exists and if not, create it
if not os.path.exists(YT_DATASTORE):
    os.makedirs(YT_DATASTORE)

# Check if sub-directory exists and if not, create it
if not os.path.exists(YT_AUDIO):
    os.makedirs(YT_AUDIO)

# Check if sub-directory exists and if not, create it
if not os.path.exists(YT_TRANSCRIPTS):
    os.makedirs(YT_TRANSCRIPTS)

# Check if sub-directory exists and if not, create it
if not os.path.exists(YT_TRANSCRIPTS_TEXT):
    os.makedirs(YT_TRANSCRIPTS_TEXT)

# Check if sub-directory exists and if not, create it
if not os.path.exists(YT_TRANSCRIPTS_WHISPER):
    os.makedirs(YT_TRANSCRIPTS_WHISPER)

# Check if sub-directory exists and if not, create it
if not os.path.exists(PODCAST):
    os.makedirs(PODCAST)

# Check if sub-directory exists and if not, create it
if not os.path.exists(PODCAST_DATASTORE):
    os.makedirs(PODCAST_DATASTORE)

# Check if sub-directory exists and if not, create it
if not os.path.exists(PODCAST_AUDIO):
    os.makedirs(PODCAST_AUDIO)

# Check if sub-directory exists and if not, create it
if not os.path.exists(PODCAST_TRANSCRIPTS):
    os.makedirs(PODCAST_TRANSCRIPTS)

## Find all David Shapiro YouTube Videos

In [ ]:
!pip install -q scrapetube
import scrapetube

In [ ]:
#All David Shapiro Videos
all_videos = scrapetube.get_search("https://www.youtube.com/@DaveShap/videos")

all_video_ids = [video['videoId'] for video in all_videos]
print (all_video_ids)

In [ ]:
# Deduplicate the lists

unique_video_ids = []
unique_video_ids = list(set(all_video_ids))
print(unique_video_ids)

## Store the list of videos in a local file for processing later

In [ ]:
# Open the file in write mode to erase existing content
with open(f'{YT_AUDIO}/videos.txt', 'w') as f:
    pass

total_videos = len(unique_video_ids)  # Get the total number of videos

for idx, video_id in enumerate(unique_video_ids, start=1):
    url = "https://www.youtube.com/watch?v=" + video_id
    print(f"Processing video {idx} of {total_videos}: {url}")
    with open(f'{YT_AUDIO}/videos.txt', 'a') as f:
        f.write(f'{video_id}\n')

### Download the audio from all the videos and store them on Google drive

In [ ]:
!pip install -q yt-dlp

In [ ]:
import os
import yt_dlp as yt

counter = 0
total_videos = len(unique_video_ids)

# Define download options
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': f'{YT_AUDIO}/clips/%(id)s.%(ext)s',
}

# Create directories if they do not exist
os.makedirs(os.path.join(YT_AUDIO, 'clips'), exist_ok=True)

for video_id in unique_video_ids:
    counter = counter + 1
    url = "https://www.youtube.com/watch?v=" + video_id
    path = os.path.join(YT_AUDIO, 'clips', f'{video_id}.webm')

    if not os.path.isfile(path):
        print(counter, "of", total_videos, ": Fetching new audio file " + video_id)

        try:
            with yt.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            with open(os.path.join(YT_AUDIO, 'processed_videos.txt'), 'a') as f:
                f.write(url + "\n")
        except:
            print(f'Video {url} is unavailable, skipping.')

    else:
        print(counter, "of", total_videos, ": Existing file: " + path)
